In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pyspark

import datetime as dt

import tabula
import joblib

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 60)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import sklearn
from sklearn.preprocessing import StandardScaler, Binarizer, LabelBinarizer, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate \
                                    ,cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,recall_score,precision_score, f1_score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.metrics import roc_curve, auc

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE

In [34]:
files = !ls raw/

In [29]:
!ls

explore_raw_data.ipynb	reports
ideas.txt		simple_random_forest.ipynb
models			simple_random_forest_w_req_amount.ipynb
pdf_to_csvs.ipynb	tsa-wait-times-january-2006-december-2015.xls
raw


In [3]:
df_0206 = pd.read_excel('raw/claims-2002-2006.xls')
df_0709 = pd.read_excel('raw/claims-2007-2009.xls')
df_1013 = pd.read_excel('raw/claims-2010-2013.xls')
df_14 = pd.read_excel('raw/claims-2014.xls')
df_15 = pd.read_excel('raw/claims-data-2015-as-of-feb-9-2016-2.xlsx')

In [36]:
df + str(5) = 'cat'

SyntaxError: can't assign to operator (<ipython-input-36-4ee3f6e7a505>, line 1)

In [40]:
for i, file in enumerate(files):
    print(i, "::::", file)

0 :::: raw/claims-2014.xls
1 :::: raw/claims-data-2015-as-of-feb-9-2016-2.xlsx


In [37]:
# files = !ls raw/
files = ['raw/claims-2014.xls', 'raw/claims-data-2015-as-of-feb-9-2016-2.xlsx']
d = {}
for i, file in enumerate(files):
    d[i] = pd.read_excel(file)
    

{0:        Claim Number Date Received       Incident Date Airport Code  \
0     2013081805991    2014-01-13 2012-12-21 00:00:00          HPN   
1     2014080215586    2014-07-17 2014-06-30 18:38:00          MCO   
2     2014010710583    2014-01-07 2013-12-27 22:00:00          SJU   
3     2014010910683    2014-01-07 2014-01-02 00:00:00          IAD   
4     2014011310783    2014-01-09 2014-01-07 00:00:00          SAT   
5     2014011610883    2014-01-13 2013-12-15 14:00:00          CHS   
6     2014012110966    2014-01-14 2013-03-24 00:00:00          SJU   
7     2014012311066    2014-01-14 2013-12-23 06:45:00          LAX   
8     2014012711166    2014-01-23 2013-12-29 14:51:00          PVD   
9     2014013011266    2014-01-27 2014-01-01 06:10:00          BLI   
10    2014020311366    2014-01-29 2014-01-23 10:15:00          LGA   
11    2014020511466    2014-02-01 2013-12-15 00:00:00          JFK   
12    2014020711566    2014-02-04 2014-01-14 05:09:00          ONT   
13    2014021311

In [41]:
df = pd.concat(d, axis=0, )

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [42]:
df.head()

Airline Name Airport Code                      Airport Name  \
0 0               USAir            HPN  Westchester County, White Plains   
  1     Delta Air Lines            MCO     Orlando International Airport   
  2              Jet Blue          SJU    Luis Munoz Marin International   
  3                 UAL            IAD   Washington Dulles International   
  4  Southwest Airlines            SAT         San Antonio International   

      Claim Number       Claim Site       Claim Type Close Amount  \
0 0  2013081805991  Checked Baggage  Personal Injury            0   
  1  2014080215586  Checked Baggage  Property Damage            0   
  2  2014010710583  Checked Baggage  Property Damage           50   
  3  2014010910683  Checked Baggage  Property Damage            0   
  4  2014011310783  Checked Baggage  Property Damage            0   

    Date Received      Disposition Incident D       Incident Date  \
0 0    2014-01-13             Deny        NaT 2012-12-21 00:00:00   
  1    2014-07-17             Deny        NaT 2014-06-30 18:38:00   
  2    2014-01-07  Approve in Full        NaT 2013-12-27 22:00:00   
  3    2014-01-07             Deny        NaT 2014-01-02 00:00:00   
  4    2014-01-09             Deny        NaT 2014-01-07 00:00:00   

                      Item Category  
0 0  Audio/Video; Jewelry & Watches  
  1                               -  
  2                    Food & Drink  
  3            Baggage/Cases/Purses  
  4          Computer & Accessories

In [11]:
for i, df in enumerate([df_0206, df_0709, df_1013, df_14, df_15]):
    print(f'DF Index {i}')
    print(df.shape)
    print('-----')
    print(df.head(1))
    print('\n')

DF Index 0
(97231, 13)
-----
  Claim Number        Date Received        Incident Date Airport Code  \
0     0909802M  2002-01-04 00:00:00  2002-12-12 00:00:00          EWR   

                   Airport Name          Airline Name       Claim Type  \
0  Newark International Airport  Continental Airlines  Property Damage   

   Claim Site   Item  Claim Amount    Status  Close Amount      Disposition  
0  Checkpoint  Other       350.000  Approved       350.000  Approve in Full  


DF Index 1
(47912, 13)
-----
    Claim Number Date Received        Incident Date Airport Code  \
0  2007010518383    2007-01-01  2006-11-25 00:00:00          PHX   

                       Airport Name        Airline Name  \
0  Phoenix Sky Harbor International  Southwest Airlines   

                Claim Type       Claim Site                     Item  \
0  Passenger Property Loss  Checked Baggage  Automobile Parts; Other   

   Claim Amount  Status  Close Amount Disposition  
0       319.730  Denied         0.0